In [1]:
import sys, os
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils

In [2]:
df=pd.read_csv(r"C:\Users\dell\Desktop\mp\fer2013.csv")

In [3]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   emotion  35887 non-null  int64 
 1   pixels   35887 non-null  object
 2   Usage    35887 non-null  object
dtypes: int64(1), object(2)
memory usage: 841.2+ KB
None


In [4]:
print(df.head())

   emotion                                             pixels     Usage
0        0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...  Training
1        0  151 150 147 155 148 133 111 140 170 174 182 15...  Training
2        2  231 212 156 164 174 138 161 173 182 200 106 38...  Training
3        4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...  Training
4        6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...  Training


In [5]:
X_train,train_y,X_test,test_y=[],[],[],[]

for index, row in df.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
           X_train.append(np.array(val,'float32'))
           train_y.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
           X_test.append(np.array(val,'float32'))
           test_y.append(row['emotion'])
    except:
        print(f"error occured at index :{index} and row:{row}")

In [6]:
num_labels = 7
epochs = 100
width, height = 48, 48

In [7]:
X_train = np.array(X_train,'float32')
train_y = np.array(train_y,'float32')
X_test = np.array(X_test,'float32')
test_y = np.array(test_y,'float32')

In [8]:
train_y=np_utils.to_categorical(train_y, num_classes=num_labels)
test_y=np_utils.to_categorical(test_y, num_classes=num_labels)

In [9]:
X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)

X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)

X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

In [10]:
##designing the cnn
#1st convolution layer
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
model.add(Conv2D(32,kernel_size= (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))


In [11]:
#2nd convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.25))

In [12]:
#3rd convolution layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

In [13]:
#fully connected neural networks
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(256))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Dense(7))
model.add(Activation('softmax'))


In [15]:
#Compliling the model
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])

In [16]:
hist = model.fit(X_train, train_y,
          batch_size=128,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, test_y),
          shuffle=True)


#Saving the  model to  use it later on
model.save_weights("fer.h5")

Epoch 1/100
225/225 [==============================] - 288s 1s/step - loss: 2.1281 - accuracy: 0.1992 - val_loss: 1.8323 - val_accuracy: 0.2522
Epoch 2/100
225/225 [==============================] - 291s 1s/step - loss: 1.7938 - accuracy: 0.2754 - val_loss: 1.6926 - val_accuracy: 0.3107
Epoch 3/100
225/225 [==============================] - 278s 1s/step - loss: 1.6498 - accuracy: 0.3411 - val_loss: 1.5489 - val_accuracy: 0.3859
Epoch 4/100
225/225 [==============================] - 275s 1s/step - loss: 1.5466 - accuracy: 0.3889 - val_loss: 1.5255 - val_accuracy: 0.3895
Epoch 5/100
225/225 [==============================] - 294s 1s/step - loss: 1.4529 - accuracy: 0.4329 - val_loss: 1.4051 - val_accuracy: 0.4581
Epoch 6/100
225/225 [==============================] - 286s 1s/step - loss: 1.3773 - accuracy: 0.4711 - val_loss: 1.2923 - val_accuracy: 0.4990
Epoch 7/100
225/225 [==============================] - 280s 1s/step - loss: 1.3035 - accuracy: 0.5020 - val_loss: 1.2632 - val_accuracy:

225/225 [==============================] - 275s 1s/step - loss: 0.3793 - accuracy: 0.8678 - val_loss: 1.4518 - val_accuracy: 0.6052
Epoch 58/100
225/225 [==============================] - 274s 1s/step - loss: 0.4388 - accuracy: 0.8477 - val_loss: 1.4439 - val_accuracy: 0.6057
Epoch 59/100
225/225 [==============================] - 277s 1s/step - loss: 0.3810 - accuracy: 0.8672 - val_loss: 1.4611 - val_accuracy: 0.6046
Epoch 60/100
225/225 [==============================] - 276s 1s/step - loss: 0.3514 - accuracy: 0.8771 - val_loss: 1.4895 - val_accuracy: 0.6127
Epoch 61/100
225/225 [==============================] - 273s 1s/step - loss: 0.3629 - accuracy: 0.8729 - val_loss: 1.5224 - val_accuracy: 0.6096
Epoch 62/100
225/225 [==============================] - 300s 1s/step - loss: 0.3452 - accuracy: 0.8816 - val_loss: 1.5067 - val_accuracy: 0.6063
Epoch 63/100
225/225 [==============================] - 327s 1s/step - loss: 0.3441 - accuracy: 0.8820 - val_loss: 1.5051 - val_accuracy: 0.608

In [17]:
train_acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']
print('Training Accuracy: ', train_acc[-1])
print('Validation Accuracy: ', val_acc[-1])

# test loss and accuracy
score, acc = model.evaluate(X_test, test_y,
                            batch_size=32)
print('Test accuracy:', acc)

Training Accuracy:  0.9201644062995911
Validation Accuracy:  0.6129841208457947
113/113 [==============================] - 7s 62ms/step - loss: 1.7124 - accuracy: 0.6130 0s - loss: 1.7356 - 
Test accuracy: 0.6129841208457947
